In [1]:
!pip install transformers
from transformers import pipeline

# classifier
classifier = pipeline(task="sentiment-analysis", model="distilbert-base-uncased-finetuned-sst-2-english")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Device set to use cpu


In [2]:
classifier("hate this")

[{'label': 'NEGATIVE', 'score': 0.9997244477272034}]

Batch Prediction

In [3]:
text_list = ["This is great",\
             "Thanks for nothing",\
             "You've got to work on your face",\
             "You're beautiful, never change"]

classifier(text_list)

[{'label': 'POSITIVE', 'score': 0.9998785257339478},
 {'label': 'POSITIVE', 'score': 0.9680057168006897},
 {'label': 'NEGATIVE', 'score': 0.8776122331619263},
 {'label': 'POSITIVE', 'score': 0.9996949434280396}]

Multiple targets

In [4]:
# if there are multiple target labels, we can return them all
classifier = pipeline(task="text-classification", model="SamLowe/roberta-base-go_emotions", top_k=None)

Device set to use cpu


In [5]:
classifier(text_list[0])

[[{'label': 'admiration', 'score': 0.9526104927062988},
  {'label': 'approval', 'score': 0.030472073704004288},
  {'label': 'neutral', 'score': 0.015236238949000835},
  {'label': 'excitement', 'score': 0.0060637635178864},
  {'label': 'gratitude', 'score': 0.005296194460242987},
  {'label': 'joy', 'score': 0.0044752066023647785},
  {'label': 'curiosity', 'score': 0.004322327207773924},
  {'label': 'realization', 'score': 0.004089601803570986},
  {'label': 'optimism', 'score': 0.0040772161446511745},
  {'label': 'disapproval', 'score': 0.004076561890542507},
  {'label': 'annoyance', 'score': 0.0035287393257021904},
  {'label': 'surprise', 'score': 0.002973065711557865},
  {'label': 'disappointment', 'score': 0.002734639449045062},
  {'label': 'love', 'score': 0.00269457814283669},
  {'label': 'amusement', 'score': 0.0024867462925612926},
  {'label': 'confusion', 'score': 0.0023607409093528986},
  {'label': 'pride', 'score': 0.0021013361401855946},
  {'label': 'sadness', 'score': 0.00177

Summarization

In [6]:
summarizer = pipeline("summarization",model="facebook/bart-large-cnn")

Device set to use cpu


In [7]:
text = """
Hugging Face is an AI company that has become a major hub for open-source machine learning.
Their platform has 3 major elements which allow users to access and share machine learning resources.
First, is their rapidly growing repository of pre-trained open-source machine learning models for things such as natural language processing (NLP), computer vision, and more.
Second, is their library of datasets for training machine learning models for almost any task.
Third, and finally, is Spaces which is a collection of open-source ML apps.

The power of these resources is that they are community generated, which leverages all the benefits of open source i.e. cost-free, wide diversity of tools, high quality resources, and rapid pace of innovation.
While these make building powerful ML projects more accessible than before, there is another key element of the Hugging Face ecosystem—their Transformers library.
"""
summarized_text = summarizer(text, min_length=5, max_length=140)[0]['summary_text']
summarized_text

'Hugging Face is an AI company that has become a major hub for open-source machine learning. They have 3 major elements which allow users to access and share machine learning resources.'

summarization + sentiment analysis

In [8]:
classifier(summarized_text)

[[{'label': 'neutral', 'score': 0.9101783633232117},
  {'label': 'approval', 'score': 0.08781366050243378},
  {'label': 'realization', 'score': 0.023256275802850723},
  {'label': 'annoyance', 'score': 0.006623789668083191},
  {'label': 'admiration', 'score': 0.0049810768105089664},
  {'label': 'disapproval', 'score': 0.004730117507278919},
  {'label': 'optimism', 'score': 0.0033590728417038918},
  {'label': 'disappointment', 'score': 0.002619005972519517},
  {'label': 'confusion', 'score': 0.001953981351107359},
  {'label': 'excitement', 'score': 0.0012417052639648318},
  {'label': 'disgust', 'score': 0.0011407802812755108},
  {'label': 'joy', 'score': 0.0010540119837969542},
  {'label': 'amusement', 'score': 0.0009572373819537461},
  {'label': 'love', 'score': 0.0008871065219864249},
  {'label': 'desire', 'score': 0.0008553271181881428},
  {'label': 'curiosity', 'score': 0.0008261064067482948},
  {'label': 'anger', 'score': 0.0007336385897360742},
  {'label': 'caring', 'score': 0.0006

Text Generation

In [9]:
chatbot = pipeline(task="text-generation", model="facebook/blenderbot-400M-distill")

Device set to use cpu


In [10]:
response = chatbot("Hi, I'm Kriti, how are you?", max_length=50)
print(response[0]["generated_text"])

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Hi, I'm Kriti, how are you? I'm okay, I guess.


Chatbot UI with Gradio

In [11]:
!pip install gradio
import gradio as gr

In [12]:
message_list = []
response_list = []

# def vanilla_chatbot(message, history):
#     response = chatbot(prompt, message, max_length=50)[0]["generated_text"]
#     message_list.append(message)
#     response_list.append(response)
#     return response

def vanilla_chatbot(message, history):
    if history is None:
        history = []
    # Build a prompt that includes previous turns
    prompt = ""
    for user_msg, bot_msg in history:
        prompt += f"User: {user_msg}\nBot: {bot_msg}\n"
    prompt += f"User: {message}\nBot: "

    # Generate a response using a longer max_length and adjusted parameters
    response = chatbot(prompt, max_length=150, temperature=0.7, top_p=0.9)[0]["generated_text"]

    # Append the latest exchange to the history
    history.append((message, response))
    return history, history

demo_chatbot = gr.ChatInterface(vanilla_chatbot, title="vanilla chatbot", description="enter text to start chatting")
demo_chatbot.launch()

/usr/local/lib/python3.11/dist-packages/gradio/chat_interface.py:334: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://fe436d109f7b0d42a2.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
